In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from pydantic import BaseModel, Field
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from AutoLLM._utils.general import load_df

In [ ]:
class ConfigItem(BaseModel):
    """
    Represents a single configuration item with 'name' and 'value' fields.
    """
    name: str = Field(..., description="The name of the configuration item.")
    description: str = Field(..., description="The value of the configuration item.")


In [16]:







class SampleConfig(BaseModel):
    """
    Represents a collection of configuration items.
    """
    configs: list[ConfigItem] = Field(
        ...,
        description="A list of configuration items, each with 'name' and 'description'."
    )


class SampleItem:

    def __init__(self, sample_config: SampleConfig, **kwargs):
        self.data = dict()
        self.field_names = [item.name for item in sample_config.configs]
        for field_name in self.field_names:
            self.data[field_name] = kwargs.get(field_name, "")

    def __repr__(self) -> str:
        return f"SampleItem(data={self.data})"



class FewShotSamples(BaseModel):
    """
    Represents a collection of samples, each with 'context', 'reasoning', and 'answer' fields.
    """
    samples: list = Field(..., description="A list of sample items.")


    def build_prompt(self):
        prompt = ""
        for sample in self.samples:
            for field in sample.field_names:
                prompt += f"{field}: {sample[field]}\n\n"
            prompt += "\n"
        return prompt
    


class InputSample(BaseModel):
    """
    Represents a collection of samples, each with 'context', 'reasoning', and 'answer' fields.
    """
    input_sample: SampleItem = Field(..., description="Input item")

    class Config:
        arbitrary_types_allowed = True

    def build_input_prompt(self):
        prompt = ""
        for field in self.input_sample.field_names:
            text = self.input_sample.data[field]
            if text:
                prompt += f"{field}: {text}\n\n" 
        return prompt
    

    def build_guide_prompt(self):
        prompt = ""
        for field in self.input_sample.field_names:
            text = self.input_sample.data[field]
            if not text:
                prompt += f"{field}:" 
        return prompt



In [7]:

df = load_df("./_data/Ecommerce/ecommerceDataset.csv")
df
# df = df[['label', 'text']].copy()
# df.to_csv('./_data/Ecommerce/ecommerceDataset.csv', index=False)

,label,text
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...
...,...,...
50424,Electronics,Strontium MicroSD Class 10 8GB Memory Card (Bl...
50425,Electronics,CrossBeats Wave Waterproof Bluetooth Wireless ...
50426,Electronics,Karbonn Titanium Wind W4 (White) Karbonn Titan...
50427,Electronics,"Samsung Guru FM Plus (SM-B110E/D, Black) Colou..."


In [4]:
ConfigItem(name="Item Description", description="Description of the item")

ConfigItem(name='Item Description', description='Description of the item')

In [17]:
sc = SampleConfig(
    configs=[
        ConfigItem(name="Item Description", description="Description of the item"),
        ConfigItem(name="Type of item", description="Answer as one of the following prompts only: {Clothing & Accessories, Electronics, Books, Household}"),
    ]
)

sample_item_list = []
for i, r in tqdm(df.iterrows(), total=len(df)):
    # print(r["text"])
    sample = SampleItem(
        sample_config=sc, 
        **{
            "Item Description": r['text'],
            "Type of item": r['label'],
        }
    )

    sample_item_list.append(sample)
df['sample_item'] = sample_item_list
df['input_text'] = df['sample_item'].apply(lambda x: InputSample(input_sample=x))
df

  0%|          | 0/50429 [00:00<?, ?it/s]

,label,text,sample_item,input_text
0,Household,Paper Plane Design Framed Wall Hanging Motivat...,SampleItem(data={'Item Description': 'Paper Pl...,input_sample=SampleItem(data={'Item Descriptio...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...","SampleItem(data={'Item Description': ""SAF 'Flo...",input_sample=SampleItem(data={'Item Descriptio...
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...,"SampleItem(data={'Item Description': ""SAF 'UV ...",input_sample=SampleItem(data={'Item Descriptio...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1...",SampleItem(data={'Item Description': 'SAF Flow...,input_sample=SampleItem(data={'Item Descriptio...
4,Household,Incredible Gifts India Wooden Happy Birthday U...,SampleItem(data={'Item Description': 'Incredib...,input_sample=SampleItem(data={'Item Descriptio...
...,...,...,...,...
50424,Electronics,Strontium MicroSD Class 10 8GB Memory Card (Bl...,SampleItem(data={'Item Description': 'Strontiu...,input_sample=SampleItem(data={'Item Descriptio...
50425,Electronics,CrossBeats Wave Waterproof Bluetooth Wireless ...,SampleItem(data={'Item Description': 'CrossBea...,input_sample=SampleItem(data={'Item Descriptio...
50426,Electronics,Karbonn Titanium Wind W4 (White) Karbonn Titan...,SampleItem(data={'Item Description': 'Karbonn ...,input_sample=SampleItem(data={'Item Descriptio...
50427,Electronics,"Samsung Guru FM Plus (SM-B110E/D, Black) Colou...",SampleItem(data={'Item Description': 'Samsung ...,input_sample=SampleItem(data={'Item Descriptio...


In [ ]:
from AutoLLM.prompts.classifier import ClassifierPrompt

instruction = 

Item Description: Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and some